# Chapter 20: Snippets of Linear Programming

In [1]:
import numpy as np
from numpy import linalg as LA
import cvxpy as cp

## Inscribed circle to a polytope

Finding the largest $\ell_p$-ball contained in a polytope $\mathcal{C} = \{ x\in\mathbb{R}^d:\langle a^{(1)},x\rangle\leq b_1,\dots, \langle a^{(n)},x\rangle\leq b_n\}$ amounts to solving the following linear program:
$$ \underset{c\in\mathbb{R}^d,r\in\mathbb{R}}{\mathrm{maximize}} \;r \quad \mbox{ subject to } \langle a^{(i)},c\rangle + r\|a^{(i)}\|_q \leq b_i, i=1,\dots,n,$$
where $q:=p/(p-1)\in[1,\infty]$ is the conjugate exponent of $p\in[1,\infty]$.

In [2]:
# Example of the d-simplex, first in the euclidean plane (p=2,d=2)
# A picture reveals that the radius of the inscribed circle is 1/(2+sqrt(2))
d = 2                       # ambient dimension
p = 2                       # index of the \ell_p-norm
q = p/(p-1)                 # conjugate index
A = np.row_stack( (np.ones((1,d)) , -np.eye(d)) )
b = np.row_stack( (1, np.zeros((d,1)) ) )
row_norms = np.zeros((d+1,1))
for i in range(d+1):
    row_norms[i] = LA.norm(A[i,:],q)
center = cp.Variable((d,1))
radius = cp.Variable(1)
objective = cp.Maximize(radius)
constraints = [A@center + radius*row_norms <= b]
inscribed_circle = cp.Problem(objective,constraints)
inscribed_circle.solve()
radius = radius.value[0]
print('In the euclidean plane, the computed inscribed radius is {:.3f}, which indeed equals 1/(2+sqrt(2))={:.3f}'
      .format(radius,1/(2+np.sqrt(2))))

In the euclidean plane, the computed inscribed radius is 0.293, which indeed equals 1/(2+sqrt(2))=0.293


When $\mathcal{C}$ is the d-simplex, it is possible to guess a formula providing the value of the radius of the inscribed $\ell_p$-ball as a function of $d$ and $p$.

In [3]:
## Compute the values of the inscribed radius as a function of d when p=2, p=Inf, and p=1
d_max = 10
P = [1,2,np.inf]
Q = [np.inf,2,1]
for k in range(3):
    p = P[k]
    q = Q[k]
    radius = np.zeros(d_max)
    for d in range(1,d_max+1):
        A = np.row_stack( (np.ones((1,d)) , -np.eye(d)) )
        b = np.row_stack( (1, np.zeros((d,1)) ) )
        row_norms = np.zeros((d+1,1))
        for i in range(d+1):
            row_norms[i] = LA.norm(A[i,:],q)
        c = cp.Variable((d,1))
        r = cp.Variable(1)
        objective = cp.Maximize(r)
        constraints = [A@c + r*row_norms <= b]
        inscribed_circle = cp.Problem(objective,constraints)
        inscribed_circle.solve()
        radius[d-1] = r.value[0]
    print('\nFor p={}, the radius of the inscribed radii when d goes from 1 to {} are:\n{}'
          .format(p,d_max,np.round(radius,4)))


For p=1, the radius of the inscribed radii when d goes from 1 to 10 are:
[0.5    0.3333 0.25   0.2    0.1667 0.1429 0.125  0.1111 0.1    0.0909]

For p=2, the radius of the inscribed radii when d goes from 1 to 10 are:
[0.5    0.2929 0.2113 0.1667 0.1382 0.1184 0.1037 0.0923 0.0833 0.076 ]

For p=inf, the radius of the inscribed radii when d goes from 1 to 10 are:
[0.5    0.25   0.1667 0.125  0.1    0.0833 0.0714 0.0625 0.0556 0.05  ]


These values strongly suggest the formula:
$$ \mbox{ radius of inscribed }\ell_p\mbox{-ball to the }d\mbox{-simplex }= \frac{1}{d^{1-1/p}(d^{1/p}+1)} = \frac{1}{d+d^{1/q}}, $$
which can be verified on a further example out of the previous range.

In [4]:
d = 13
p = 5
q = p/(p-1)
A = np.row_stack( (np.ones((1,d)) , -np.eye(d)) )
b = np.row_stack( (1, np.zeros((d,1)) ) )
row_norms = np.zeros((d+1,1))
for i in range(d+1):
    row_norms[i] = LA.norm(A[i,:],q)
center = cp.Variable((d,1))
radius = cp.Variable(1)
objective = cp.Maximize(radius)
constraints = [A@center + radius*row_norms <= b]
inscribed_circle = cp.Problem(objective,constraints)
inscribed_circle.solve()
radius = radius.value[0]
print('The computed inscribed radius is {:.3f}, which indeed equals 1/(d+d^{{1/q}})={:.3f}'
      .format(radius,1/(d+d**(1/q))))

The computed inscribed radius is 0.048, which indeed equals 1/(d+d^{1/q})=0.048


## Proximity between two polytopes

Finding the "proximity", relatively to the $\ell_\infty$-norm, between the polytopes $\mathcal{C}=\{ x\in\mathbb{R}^d: Ax\leq b\}$ and $\mathcal{C}'=\{ x\in\mathbb{R}^d: A'x\leq b'\}$ amounts to solving the following linear program program
$$ \underset{x,x'\in\mathbb{R}^d,c\in\mathbb{R}}{\mathrm{minimize}} \;c \quad \mbox{subject to } Ax\leq b, A'x'\leq b', -c\leq x-x'\leq c.$$

In [5]:
# Example of the d-simplex and a shifted version of it
d = 13
A = np.row_stack( (np.ones((1,d)), -np.eye(d)) )
b = np.row_stack( (1, np.zeros((d,1))) )
shift = 2*np.random.rand(d,1)/d
AA = A
bb = b + A@shift
x = cp.Variable((d,1))
xx = cp.Variable((d,1))
c = cp.Variable(1)
objective = cp.Minimize(c)
constraints = [ A@x <= b]
constraints+= [ AA@xx <= bb]
constraints+= [ x-xx <= c]
constraints+= [ xx-x <= c]
proximity = cp.Problem(objective,constraints)
proximity.solve()
proxy = c.value[0]
print('proxy={:.4e}'.format(proxy))

proxy=1.8908e-02


When $\mathcal{C}$ is the $d$-simplex and $\mathcal{C}'$ is the shifted version $\mathcal{C}' = s+\mathcal{C}$ with $s=t[1;\dots;1]$ for $t>0$, experiments would reveal that the "proximity" between $\mathcal{C}$ and $\mathcal{C}'$ is $\max\{t-1/d,0\}$, as confirmed by the following test. 

In [6]:
d = np.random.randint(50)
t = (1/2+np.random.random()*3/2)/d
A = np.row_stack( (np.ones((1,d)), -np.eye(d)) )
b = np.row_stack( (1, np.zeros((d,1))) )
shift = t*np.ones((d,1))
AA = A
bb = b + A@shift
x = cp.Variable((d,1))
xx = cp.Variable((d,1))
c = cp.Variable(1)
objective = cp.Minimize(c)
constraints = [ A@x <= b]
constraints+= [ AA@xx <= bb]
constraints+= [ x-xx <= c]
constraints+= [ xx-x <= c]
proximity = cp.Problem(objective,constraints)
proximity.solve()
proxy = c.value[0]
print('The computed "proximity" is {:.3f}, which indeed equals max{{t-1,0}}/d={:.3f}'.format(proxy,max(t-1/d,0)))

The computed "proximity" is 0.003, which indeed equals max{t-1,0}/d=0.003


## Hausdorff distance between two bounded polytopes

Finding the Hausdorff distance, relatively to the $\ell_1$-norm, between the polytopes $\mathcal{C} = \{ x\in\mathbb{R}^d:Ax\leq b\}$ and  $\mathcal{C}' = \{ x\in\mathbb{R}^d:A'x\leq b'\}$ amounts to solving $K+K'$ linear programs, where $K$ and $K'$ are the numbers of vertices of $\mathcal{C}$ and of $\mathcal{C}'$, respectively. Denoting these vertices by $v_1,\dots,v_k$ and $v'_1,\dots,v'_{K'}$, the linear programs are:
$$ \underset{x',c\in\mathbb{R}^d}{\mathrm{minimize}} \; \sum_{j=1}^d c_j \mbox{ subject to } A'x'\leq b', -c\leq v_k-x'\leq c; $$
$$ \underset{x,c\in\mathbb{R}^d}{\mathrm{minimize}} \; \sum_{j=1}^d c_j \mbox{ subject to } Ax\leq b, -c\leq v'_k-x\leq c. $$

In [7]:
# Example of the d-simplex and a shifted version of it
d = 13
A = np.row_stack( (np.ones((1,d)), -np.eye(d)) )
b = np.row_stack( (1, np.zeros((d,1))) )
V = np.column_stack( (np.zeros((d,1)), np.eye(d)) )    #the columns of V are vertices of the simplex
shift = 2*np.random.rand(d,1)/d
AA = A
bb = b + A@shift
VV = shift + V                                         #the columns of VV are vertices of the shifted simplex
left_terms = np.zeros(d+1)
right_terms = np.zeros(d+1)
for k in range(d+1):
    xx = cp.Variable((d,1))
    c = cp.Variable((d,1))
    objective = cp.Minimize(cp.sum(c))
    constraints = [ AA@xx <= bb ]
    constraints+= [ V[:,[k]]-xx <= c]
    constraints+= [ xx-V[:,[k]] <= c]
    first_linear = cp.Problem(objective,constraints)
    first_linear.solve()
    left_terms[k] = first_linear.value
    x = cp.Variable((d,1))
    c = cp.Variable((d,1))
    objective = cp.Minimize(cp.sum(c))
    constraints = [ A@x <= b ]
    constraints+= [ VV[:,[k]]-x <= c]
    constraints+= [ x-VV[:,[k]] <= c]
    second_linear = cp.Problem(objective,constraints)
    second_linear.solve()
    right_terms[k] = second_linear.value
Haus_dist = max(max(left_terms),max(right_terms))  
print('Haus_dist={:.4f}'.format(Haus_dist))

Haus_dist=1.0619


When $\mathcal{C}$ is the $d$-simplex and $\mathcal{C}'$ is the shifted version $\mathcal{C}' = s+\mathcal{C}$ with $s=t[1;\dots;1]$ for $t>0$, experiments would reveal that the Hausdorff distance between $\mathcal{C}$ and $\mathcal{C}'$ is $d$ $t$, as confirmed by the following test.

In [8]:
d = np.random.randint(15)
t = np.random.random()
A = np.row_stack( (np.ones((1,d)), -np.eye(d)) )
b = np.row_stack( (1, np.zeros((d,1))) )
V = np.column_stack( (np.zeros((d,1)), np.eye(d)) )    #the columns of V are vertices of the simplex
shift = t*np.ones((d,1))
AA = A
bb = b + A@shift
VV = shift + V                                         #the columns of VV are vertices of the shifted simplex
left_terms = np.zeros(d+1)
right_terms = np.zeros(d+1)
for k in range(d+1):
    xx = cp.Variable((d,1))
    c = cp.Variable((d,1))
    objective = cp.Minimize(cp.sum(c))
    constraints = [ AA@xx <= bb ]
    constraints+= [ V[:,[k]]-xx <= c]
    constraints+= [ xx-V[:,[k]] <= c]
    first_linear = cp.Problem(objective,constraints)
    first_linear.solve()
    left_terms[k] = first_linear.value
    x = cp.Variable((d,1))
    c = cp.Variable((d,1))
    objective = cp.Minimize(cp.sum(c))
    constraints = [ A@x <= b ]
    constraints+= [ VV[:,[k]]-x <= c]
    constraints+= [ x-VV[:,[k]] <= c]
    second_linear = cp.Problem(objective,constraints)
    second_linear.solve()
    right_terms[k] = second_linear.value
Haus_dist = max(max(left_terms),max(right_terms))  
print('The computed Hausdorff distance is {:.3f}, which indeed equals dt={:.3f}'.format(Haus_dist,d*t))

The computed Hausdorff distance is 3.707, which indeed equals dt=3.707
